Introduction:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ripser import ripser
from persim import plot_diagrams

import circle_bundles as cb

First, get a large sample of optical patches from the Sintel dataset:

In [ ]:
import pickle 
import pandas as pd

patches_per_frame = 400
folder_path = ".../MPI-Sintel-complete/training/flow"   #path to Sintel flow frames

patch_df, file_paths = get_patch_sample(
    folder_path,
    patches_per_frame = patches_per_frame,
    d = 3)

print('')
print(f'{len(patch_df)} optical flow patches sampled')

#Downsample if necessary
max_samples = 400000
if len(patch_df) > max_samples:
    patch_df = patch_df.sample(n=max_samples)


Next, preprocess the data -- compute contrast norms and keep only patches with contrast norm in the top 20%.  Then, use a KNN density estimator to measure the density of dataset near each datapoint:

In [ ]:
hc_frac = 0.2
max_samples = 50000
k = [300]

print('Preprocessing data...')
patch_df = preprocess_flow_patches(
    patch_df,
    hc_frac = hc_frac,
    max_samples = max_samples,
    k_list = k)

print('Preprocessing complete.')

Finally, keep only the top 50% of patches by density value:

In [ ]:
p = 0.5 
n_samples = int(p*len(patch_df))
data = np.vstack(patch_df['patch'])[:n_samples]    #Data is already sorted in decreasing order by density
print(f'Downsampled to {len(data)} patches')

Compute a persistence diagram from a sample of the dataset.  If the dataset is truly concentrated around the proposed torus model, we would expect to see two 1-dimensional persistent classes and a persistent class in dimension 2: 

In [ ]:
diagrams = ripser(data, maxdim = 2, n_perm = 500)['dgms']
plot_diagrams(diagrams, show=True)    

Observe that we see a single 1-dimensional persistent class and no persistent classes in dimension 2.  

Proceed with local-to-global analysis.  Compute the predominant flow axis in $\mathbb{RP}^{1}$ (as introduced by Adams et al.) and construct a cover of $\mathbb{RP}^{1}$:

In [ ]:
predom_dirs, ratios = get_predominant_dirs(data)    #compute directionalities for later use

#Construct a cover of the base space
n_landmarks = 16
landmarks = np.linspace(0, np.pi, n_landmarks, endpoint= False)
overlap = 1.99
radius = overlap* np.pi/(2*n_landmarks)

cover = MetricBallCover(predom_dirs, landmarks, radius, metric = cb.RP1AngleMetric())
cover_data = cover.build()

#Show a summary of the construction
summ = cover.summarize(plot = True)

View a sample of the dataset arranged by predominant flow direction:

In [ ]:
n_samples = 8

label_func = [fr"$\theta = {np.round(pred/np.pi, 2)}$" + r"$\pi$" for pred in predom_dirs]
fig = show_data_vis(
    data, 
    patch_vis, 
    label_func = label_func, 
    angles = predom_dirs, 
    sampling_method = 'angle', 
    max_samples = n_samples)
plt.show()


$\textbf{Main bundle construction:}$ Construct local circular coordinates, compute approximate transition matrices and characteristic classes:


In [ ]:
bundle = build_bundle(
    data,
    cover,
    show=True,
)


View the correlations between local circular coordinates on overlaps:

In [ ]:
fig = bundle.compare_trivs(ncols = 4)
plt.show()

Now, set up a bundle with just the high-directionality data:

In [ ]:
thresh = 0.8
high_inds = ratios > thresh

print(f'{np.sum(high_inds)} high-directionality patches')

high_cover = MetricBallCover(predom_dirs[high_inds], landmarks, radius, metric = rp1_metric())
high_cover_data = high_cover.build()
high_bundle = build_bundle(
    data[high_inds],
    high_cover,
    show=True,
)


Get global toroidal coordinates for the high-directionality data:

In [ ]:
high_triv_result = high_bundle.get_global_trivialization()


Show a sample of coordinatized patches:

In [ ]:
per_row = 5
per_col = 9
coords = np.array([predom_dirs[high_inds], high_triv_result.F]).T

fig = lattice_vis(
    high_bundle.data,
    coords,
    patch_vis,
    per_row=per_row,
    per_col = per_col,
    figsize=19,
    thumb_px=350,   
    dpi=350, 
    padding = 0,
)

plt.show()


Get a visualization of the coordinatized low-directionality data:

In [ ]:
thresh = 0.7
low_inds = ratios < thresh
print(f'{np.sum(low_inds)} low-directionality patches')
low_data = data[low_inds]


per_row = 5
per_col = 9
coords = np.array([predom_dirs[low_inds], triv_result.F[low_inds]]).T

fig = lattice_vis(
    low_data,
    coords,
    patch_vis,
    per_row=per_row,
    per_col = per_col,
    figsize=19,
    thumb_px=350,   
    dpi=350,
    padding = 0
)

plt.show()

